In [5]:
import pandas as pd
from IPython.display import display
import folium

path_jo = '/Users/gordonwatt/code/dev/munro/v2/data/Jo.csv'
path_gordon = '/Users/gordonwatt/code/dev/munro/v2/data/Gordon.csv'
path_alan = '/Users/gordonwatt/code/dev/munro/v2/data/Alan_Dunmur.csv'
path_ian = '/Users/gordonwatt/code/dev/munro/v2/data/Ian_Sharp.csv'
path_iain = '/Users/gordonwatt/code/dev/munro/v2/data/Iain_Young.csv'
path_freya = '/Users/gordonwatt/code/dev/munro/v2/data/freya.csv'

# Load the CSV files
file1 = pd.read_csv(path_alan, encoding='utf-8')
file2 = pd.read_csv(path_gordon, encoding='utf-8')
  
# Drop rows where ascents = true (we only want unclimbed hills)
file1_filtered = file1[file1['Ascent'].notna()]
file2_filtered = file2[file2['Ascent'].isna()]

# Define the column to compare and merge the filtered DataFrames on the selected column
column_name = 'Identifier'
common_rows = pd.merge(file1_filtered, file2_filtered, on=column_name)

# Remove duplicate columns from merge
common_rows = common_rows[[col for col in common_rows.columns if not col.endswith('_y')]]

# Extract the numeric part of 'Section_x' and store it in a new column 'Section' (this deals with section names like 10B)
common_rows['Section_x'] = common_rows['Section_x'].str.extract(r'(\d+)').astype(int)

# Sort data by Section, then height
common_rows = common_rows.sort_values(by=['Section_x', 'Area/Region_x', 'Metres_x'])

# Remove _x suffix from headings
common_rows.columns = common_rows.columns.str.replace('_x$', '', regex=True)

# Drop columns for output
output_common_rows = common_rows.drop(columns=['Identifier', 'Parent', 'Ascent', 'Notes'])

# display(common_rows.tail(10))
display(output_common_rows.head(20))

,Name,Area/Region,Section,Metres,Feet,Latitude,Longitude
0,Beinn a' Chleibh,Inveraray to Crianlarich,1,916.3,3006,56.390230,-4.835630
5,Beinn Narnain,Inveraray to Crianlarich,1,926.8,3041,56.220956,-4.789003
8,Ben Vorlich,Inveraray to Crianlarich,1,942.8,3093,56.274021,-4.755046
2,Beinn Bhuidhe,Inveraray to Crianlarich,1,948.5,3112,56.326686,-4.906709
4,Beinn Dubhchraig,Inveraray to Crianlarich,1,978.6,3211,56.391295,-4.743193
6,Ben Lui,Inveraray to Crianlarich,1,1131.4,3712,56.397004,-4.810509
3,Beinn Chabhair,Loch Lomond to Strathyre,1,932.2,3058,56.325779,-4.641476
1,Beinn a' Chroin,Loch Lomond to Strathyre,1,941.4,3089,56.331991,-4.609644
9,Stob Binnein,Loch Lomond to Strathyre,1,1165.0,3822,56.370717,-4.535753
7,Ben More,Loch Lomond to Strathyre,1,1174.0,3852,56.385952,-4.540087


In [6]:
# Save the result to a new CSV file
# output_common_rows.to_csv('IY_GRW.csv', index=False)

In [7]:
# Draw a map

# Create a map centered around the mean latitude and longitude
center_lat = common_rows['Latitude'].mean()
center_lon = common_rows['Longitude'].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=7)

# Add markers with name labels next to each point
for idx, row in output_common_rows.iterrows():
    # Customize marker icon and color
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['Name'],  # Display the name when clicked
        # tooltip=row['Name'],  # Display the name on hover
        icon=folium.Icon(color='blue', icon='info-sign')  # Customize icon color and style
    ).add_to(m)

# Display the map in a Jupyter Notebook
m


In [8]:
# save map
m.save("/Users/gordonwatt/code/dev/morningglass.github.io/data/APD_NOT_GRW.html")